In [1]:
# Paruošia pūpsnių indeksus ML mokymui ir vertinimui.
# 
# Panaudojant pacientų įrašų vardų SubjCode sąrašus train_lst, validate_lst, test_lst iš disko,
# sukūriami atitinkami indeksų failai train_ind_lst, validate_ind_lst, test_ind_lst ir įrašomi į diską.
# Kartu su duomenimis ir all_beats_attr, jie naudojami klasifikatorių kūrimui. 
# 
# Nebalansuotas variantas.
# 
# Paruošti pūpsnių indeksų masyvai gali būti panaudoti tiesioginiam sekų 
# formavimui iš EKG įrašų (rec) skaičiavimo metu arba sekų Numpy masyvų formavimui
#  
# //////////////////////////////////////////////////////////////////////////////////////////////////////
#
import pandas as pd
import numpy as np
from numpy import loadtxt, savetxt
import matplotlib.pyplot as plt
from pathlib import Path
from icecream import ic
import json, time, sys

from zive_util_vu import create_dir
# from zive_util_vu import anotaciju_pasiskirstymas_v2, runtime
from zive_util_vu import runtime
from zive_util_vu import get_annotations_table, print_annotations_table
from collections import Counter
 

def get_subj_count(SubjCodes):
    """
    input: SubjCodes: list of int
    output: count:int
    """
    userNrs = [int(SubjCode/10) for SubjCode in SubjCodes]
    items = Counter(userNrs).keys()
    return len(items)

my_os=sys.platform

print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

#  MIT2ZIVE duomenų aplankas
db_folder = 'DUOM_VU'

# Aplankas su MIT2ZIVE EKG įrašais (.npy) ir anotacijomis (.json)
rec_folder = 'records_npy'

# Duomenų schemos aplanko pavadinimas
# SCHEMA_FOLDER = 'schema_NSV_50_nbal'

# Kas kiek išvedamas apdorotų duomenų skaičius
show_period = 100

# Anotacijoms priskirtos klasės
selected_beats = {'N':0, 'S':1, 'V':2, 'U':3}

# Sekos kanalų skaičius
n_channels = 1

# Kiekis pūpsnių, kurie atmetami EKG įrašo pradžioje ir pabaigoje
beats_skiped = 10

# ///////////////////////////////////////////////////////////////////////////////

# Anotacijų sąrašas
annot_list = [key for key in selected_beats]

# swapped_selected_beats
swapped_selected_beats = {v: k for k, v in selected_beats.items()}

# Nuoroda į duomenų aplanką
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su  EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, rec_folder)

# Nuskaitome ir suformuojame DS1Z
file_path = Path(rec_dir, 'train_subjcode_lst.csv')
DSZ1 = list(np.loadtxt(file_path, delimiter=',', dtype="int"))
# print("DSZ1:", DSZ1)

# Nuskaitome ir suformuojame DS1Z
file_path = Path(rec_dir, 'validation_subjcode_lst.csv')
DSZ2 = list(np.loadtxt(file_path, delimiter=',', dtype="int"))
# print("DSZ2:", DSZ2)

# Nuskaitome ir suformuojame DS3Z
file_path = Path(rec_dir, 'test_subjcode_lst.csv')
DSZ3 = list(np.loadtxt(file_path, delimiter=',', dtype="int"))
# print("DSZ3:", DSZ3)

# Nuoroda į duomenų schemos aplanką ir vertinimo imties sekų indeksų failas
all_beats_attr_fname = 'all_beats_attr_z.csv'

# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create_z.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)

fs = info_create['fs'] # diskretizavimo dažnumas
SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas

# Klasių skaičius
n_classes = len(selected_beats)

print("\nParuošia EKG pūpsnių indeksus ML mokymui ir vertinimui")
print("Nebalansuotas variantas")

print("\nBendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("Zive EKG įrašų aplankas: ", db_folder)

print("Diskretizavimo dažnis: ", fs)
print("\nIš viso pacientų:",get_subj_count(SubjCodes))
print("Pacientų įrašų sąrašas:\n",SubjCodes)

print("\nPacientų saraše mokymui DSZ1 :", get_subj_count(DSZ1))
print("Pacientų sąrašas mokymui DSZ1:", DSZ1)

print("\nPacientų saraše validavimui DSZ2 :", get_subj_count(DSZ2))
print("Pacientų sąrašas validavimui DSZ2:", DSZ2)

print("\nPacientų saraše testavimui DSZ3 :", get_subj_count(DSZ3))
print("Pacientų sąrašas testavimui DSZ3:", DSZ3)

print("\nECK įrašų aplankas: ", rec_dir)
print('Klasifikavimo schema:', selected_beats)
print('Klasių skaičius:', len(selected_beats))
print('Pūpsnių atributų masyvo failas:', all_beats_attr_fname)

print("\nParametrai pūpsnių kiekiui apriboti")
print(f"beats_skiped: {beats_skiped}")
print(f"Kiekviename EKG įraše praleisti {beats_skiped} pirmi ir {beats_skiped} paskutiniai pūpsniai")



OS in my system :  linux

Paruošia EKG pūpsnių indeksus ML mokymui ir vertinimui
Nebalansuotas variantas

Bendras Zive duomenų aplankas:  /home/kesju/DI/Data/MIT&ZIVE/VU
Zive EKG įrašų aplankas:  DUOM_VU
Diskretizavimo dažnis:  200

Iš viso pacientų: 87
Pacientų įrašų sąrašas:
 [10000, 10001, 10002, 10010, 10011, 10012, 10013, 10014, 10015, 10020, 10021, 10022, 10023, 10024, 10025, 10026, 10030, 10031, 10040, 10041, 10042, 10050, 10051, 10052, 10053, 10054, 10055, 10056, 10057, 10058, 10059, 10060, 10061, 10062, 10063, 10070, 10071, 10080, 10081, 10082, 10083, 10084, 10085, 10086, 10087, 10088, 10089, 10090, 10091, 10092, 10093, 10094, 10095, 10096, 10097, 10098, 10099, 10100, 10101, 10110, 10111, 10112, 10120, 10121, 10122, 10130, 10131, 10132, 10133, 10134, 10135, 10136, 10137, 10138, 10139, 10140, 10150, 10151, 10152, 10153, 10154, 10155, 10156, 10157, 10158, 10159, 10160, 10161, 10170, 10180, 10181, 10182, 10183, 10184, 10185, 10186, 10187, 10188, 10190, 10191, 10192, 10193, 10194,

In [19]:

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns", 19)
pd.set_option('display.width', 1000)

cols_pattern = ['N', 'S', 'V', 'U']
 
#  from mit2zive_util import create_set_from_rec
from sklearn.preprocessing import StandardScaler
from numpy import asarray
from zive_util_vu import split_SubjCode

start_time = time.time()

# Nuskaitome pūpsnių atributų freimą iš rec_dir aplanko
file_path = Path(rec_dir, all_beats_attr_fname)
all_beats_attr = pd.read_csv(file_path, index_col=0)

# Visų sekų indeksų sąrašas
all_beat_indices = all_beats_attr.index
# print(all_beat_indices)
# DS = DS1 + DS2
# all_ind_lst = []
# for userNr in DS:
#     selected_ind = all_beat_indices[all_beats_attr['userNr']==userNr]
#     all_ind_lst.extend(selected_ind.to_list())

print("\nAnotacijų pasiskirstymas visuose duomenyse\n")
labels_table, labels_sums = get_annotations_table(all_beats_attr, cols_pattern=cols_pattern)
print_annotations_table(labels_table, labels_sums, Flag1=False, Flag2=False)
count = len(DSZ1) + len(DSZ2) + len(DSZ3)
print(f"Bendras įrašų skaičius: {count}")

# Suformuojame sekų masyvus klasifikatoriaus mokymui ir vertinimui

# MOKYMO IMTIS

# Suformuojame imčių sekų indeksų sąrašus. Formuodami sąrašus
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus

train_ind_lst = []
for SubjCode in DSZ1:
    userNr, recordingNr = split_SubjCode(SubjCode)
    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recordingNr)]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    selected_ind_lst = selected_ind.to_list()
    # print("selected_ind_lst parameters:")
    # print(f"SubjCode: {SubjCode}  first elem: {selected_ind_lst[0]} last elem: {selected_ind_lst[-1]}  tot: {len(selected_ind_lst)}")
    train_ind_lst.extend(selected_ind_lst)

# Anotacijų pasiskirstymas vertinimo duomenyse
print("\n")
print("\nAnotacijų pasiskirstymas mokymo duomenyse\n")
labels_table, labels_sums = get_annotations_table(all_beats_attr, ind_lst= train_ind_lst, cols_pattern=cols_pattern)
print_annotations_table(labels_table, labels_sums, Flag1=False, Flag2=False)

# Įrašome mokymo imties sekų indeksų sąrašą į diską
file_path = Path(rec_dir, 'train_ind_lst.csv')
np.savetxt(file_path, np.array(train_ind_lst), delimiter=',', fmt='%d')
print("\nMokymo imties sekų indeksų sąrašas įrašytas į: ", file_path )
print(f"Įrašų skaičius mokymo imtyje: {len(DSZ1)}")

# Suformuojame mokymo sekų ir jų klasių numerių masyvus
# X_train, y_train = create_set_from_rec(rec_dir, all_beats_attr, train_ind_lst, window_left_side, window_right_side)
# print("\nMokymo imties dimensijos")
# print(f"{X_train.shape=} {y_train.shape=}")

# Įrašome mokymui skirtus masyvus į npz file diske
# file_path = Path(SCHEMA_PATH, 'train_set.npz')
# np.savez_compressed(file_path, X=X_train, y=y_train)
# print("\nMokymo sekų ir jų klasių numerių masyvas įrašytas: į ", file_path )


# VALIDAVIMO IMTIS

validation_ind_lst = []
for SubjCode in DSZ2:
    userNr, recordingNr = split_SubjCode(SubjCode)
    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recordingNr)]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    selected_ind_lst = selected_ind.to_list()
    # print("selected_ind_lst parameters:")
    # print(f"SubjCode: {SubjCode}  first elem: {selected_ind_lst[0]} last elem: {selected_ind_lst[-1]}  tot: {len(selected_ind_lst)}")
    validation_ind_lst.extend(selected_ind_lst)

# Anotacijų pasiskirstymas vertinimo duomenyse
print("\n")
print("\nAnotacijų pasiskirstymas validavimo duomenyse\n")
labels_table, labels_sums = get_annotations_table(all_beats_attr, ind_lst= validation_ind_lst, cols_pattern=cols_pattern)
print_annotations_table(labels_table, labels_sums, Flag1=False, Flag2=False)

# Įrašome validavimo imties sekų indeksų sąrašą į diską
file_path = Path(rec_dir, 'validation_ind_lst.csv')
np.savetxt(file_path, np.array(validation_ind_lst), delimiter=',', fmt='%d')
print("\nValidavimo validavimo sekų indeksų sąrašas įrašytas į: ", file_path )
print(f"Įrašų skaičius validavimo imtyje: {len(DSZ2)}")


# TESTINĖ IMTIS

test_ind_lst = []
for SubjCode in DSZ3:
    userNr, recordingNr = split_SubjCode(SubjCode)
    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recordingNr)]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    selected_ind_lst = selected_ind.to_list()
    # print("selected_ind_lst parameters:")
    # print(f"SubjCode: {SubjCode}  first elem: {selected_ind_lst[0]} last elem: {selected_ind_lst[-1]}  tot: {len(selected_ind_lst)}")
    test_ind_lst.extend(selected_ind_lst)

# Anotacijų pasiskirstymas vertinimo duomenyse
print("\n")
print("\nAnotacijų pasiskirstymas testiniuose duomenyse\n")
labels_table, labels_sums = get_annotations_table(all_beats_attr, ind_lst= test_ind_lst, cols_pattern=cols_pattern)
print_annotations_table(labels_table, labels_sums, Flag1=False, Flag2=False)

# Įrašome testinės imties sekų indeksų sąrašą į diską
file_path = Path(rec_dir, 'test_ind_lst.csv')
np.savetxt(file_path, np.array(test_ind_lst), delimiter=',', fmt='%d')
print("\nTestinės imties sekų indeksų sąrašas įrašytas į: ", file_path )
print(f"Įrašų skaičius testinėje imtyje: {len(DSZ3)}")

end_time = time.time()
print('\n')
runtime(end_time-start_time)

# Suformuojame info_prep masyvą ir įrašome į diską

info = {
    'fs': fs,
    'SubjCodes':SubjCodes,
    'selected_beats':selected_beats,
    'beats_skiped':beats_skiped
    }

file_path = Path(rec_dir,'info_prep_tst.json')
with open(file_path, 'w') as f:
    json.dump(info, f)
    
print("\nSchemos parametrai info.json yra aplanke: ", file_path, "\n" )

print("\nPabaiga.............")



Anotacijų pasiskirstymas visuose duomenyse

'N': 47935, 'S': 177, 'V': 235, 'U': 10
Total:  48357
Bendras įrašų skaičius: 57



Anotacijų pasiskirstymas mokymo duomenyse

'N': 17573, 'S': 104, 'V': 76, 'U': 5
Total:  17758

Mokymo imties sekų indeksų sąrašas įrašytas į:  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU_TST\records_npy\train_ind_lst.csv
Įrašų skaičius mokymo imtyje: 21



Anotacijų pasiskirstymas validavimo duomenyse

'N': 9581, 'S': 25, 'V': 3
Total:  9609

Validavimo validavimo sekų indeksų sąrašas įrašytas į:  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU_TST\records_npy\validation_ind_lst.csv
Įrašų skaičius validavimo imtyje: 16



Anotacijų pasiskirstymas testiniuose duomenyse

'N': 20667, 'S': 48, 'V': 156, 'U': 5
Total:  20876

Testinės imties sekų indeksų sąrašas įrašytas į:  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU_TST\records_npy\test_ind_lst.csv
Įrašų skaičius testinėje imtyje: 20


Runtime: 00:00:00

Schemos parametrai info.json yra aplanke:  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU_TST\records_npy\info_prep_